<a href="https://colab.research.google.com/github/Navifra-Denny/01_linear_algegra/blob/main/01.pca/principal_component_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [48]:
SLOPE_MIN = -50.0
SLOPE_MAX = 50.0

def generate_clipped_gaussian(mean, std, min_val, max_val, size):
    data = np.random.normal(mean, std, size)
    # Clip to specified range
    data = np.clip(data, min_val, max_val)
    # Round to 1 decimal place
    return np.round(data, 1)

In [49]:
num_samples = 300

# Gaussian parameters
height_mean, height_std = 170, 6.8
weight_mean, weight_std = 80, 13

# Generate height and weight with clipping and rounding
heights = generate_clipped_gaussian(height_mean, height_std, 0, 210, num_samples)
weights = generate_clipped_gaussian(weight_mean, weight_std, 40, 150, num_samples)

# Stack into (N, 2) dataset
dataset = np.column_stack((heights, weights))

# Compute means
height_avg = np.mean(heights)
weight_avg = np.mean(weights)


## SS (Sumo of Squares)

![ss](https://raw.githubusercontent.com/Navifra-Denny/01_linear_algegra/main/01.pca/img.png)


> 모든 데이터에서 원점(평균으로 이동된 점)을 지나는 직선에 수선을 발을 내리면, 원점으로부터 수선의 발까지의 길이를 구할 수 있게 된다. 원점을 지나는 직선의 기울기가 변함에 따라, 이 빨간선들의 길이 또한 변하게 된다. PCA 에서는 이 빨간선들의 길이 제곱들의 합이 최대가 되는 직선을 찾는다. 이것이 SS(Sum of Squares) 이다.


## 요약
* 직선은 평균점을 지나며, 기울기 𝑚 에 따라 정의됨:
$$ y = m(x - \bar{x}) + \bar{y} $$
* 각 데이터 포인트 $(x_i, y_i)$ 에 대해 직선에 내린 **수선의 발**을 구하고,
* 그 **수선 길이의 제곱**을 계산한 후, 전체에 대한 합산:
$$ SS(m) = \sum_{i=1}^{N} d_i^2 $$


### 직선과 점 사이의 거리 계산 공식
* 직선: $y=mx+b$
* 점: $(x_0, y_0)$
* 직선과 점 사이 거리:
$$ d = \frac{\left| mx_0 - y_0 + b \right|}{\sqrt{m^2 + 1}} $$


In [50]:
# calculate SS function
def calculate_ss(x, y, slope, x0, y0):
    """모든 점에서 평균점을 지나는 직선으로 내린 수선의 길이 제곱합(SS) 계산"""
    # 직선: y = m(x - x0) + y0 → y = mx + (y0 - m*x0) → b 계산
    b = y0 - slope * x0
    numerator = np.abs(slope * x - y + b)
    denominator = np.sqrt(slope**2 + 1)
    distances = numerator / denominator
    ss = np.sum(distances**2)
    return ss

In [52]:
# ===== PCA 1st 주성분 기울기 계산 함수 =====
def calculate_pca_slope(x, y):
    data = np.column_stack((x, y))
    data_centered = data - np.mean(data, axis=0)
    cov = np.cov(data_centered, rowvar=False)
    eigvals, eigvecs = np.linalg.eigh(cov)
    principal_vec = eigvecs[:, np.argmax(eigvals)]
    slope = principal_vec[1] / principal_vec[0]
    return slope

# 실제 PCA 주성분 기울기 계산
optimal_m = calculate_pca_slope(heights, weights)

In [53]:
# 업데이트 함수 정의
def update_line(m):
    plt.figure(figsize=(8, 6))
    plt.scatter(heights, weights, color='blue', alpha=0.7, edgecolors='black', label="Samples")

    # 평균 축
    plt.axvline(x=height_avg, color='red', linestyle='--', linewidth=0.5, label=f'Height Mean = {height_avg:.1f} cm')
    plt.axhline(y=weight_avg, color='red', linestyle='--', linewidth=0.5, label=f'Weight Mean = {weight_avg:.1f} kg')

    # 유저 기울기 직선
    x_vals = np.linspace(min(heights), max(heights), 100)
    y_vals = m * (x_vals - height_avg) + weight_avg
    plt.plot(x_vals, y_vals, color='green', linestyle='-', linewidth=2, label=f'Slope = {m:.2f}')

    # PCA 주성분 방향 (진짜)
    y_opt_vals = optimal_m * (x_vals - height_avg) + weight_avg
    plt.plot(x_vals, y_opt_vals, color='orange', linestyle='--', linewidth=2, label=f'PCA Slope = {optimal_m:.2f}')

    plt.title("Synthetic Dataset: Height vs Weight")

    # 보기 좋게 축 고정
    margin_x = 10
    margin_y = 10
    plt.xlim(height_avg - 3 * height_std - margin_x, height_avg + 3 * height_std + margin_x)
    plt.ylim(weight_avg - 3 * weight_std - margin_y, weight_avg + 3 * weight_std + margin_y)

    plt.xlabel("Height (cm)")
    plt.ylabel("Weight (kg)")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [54]:
# 슬라이더: 기울기 m (-5 ~ 5 사이 실수)
slope_slider = widgets.FloatSlider(
    value=0.0,
    min=SLOPE_MIN,
    max=SLOPE_MAX,
    step=0.1,
    description='Slope (m):',
    continuous_update=True,
    orientation='horizontal',
    readout=True
)

# 슬라이더와 함수 연결
widgets.interact(update_line, m=slope_slider);

interactive(children=(FloatSlider(value=0.0, description='Slope (m):', max=50.0, min=-50.0), Output()), _dom_c…